<a href="https://colab.research.google.com/github/hijera/NN_colab_hw/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F%C2%BB_(%D0%92%D0%BB%D0%B0%D1%81%D0%BE%D0%B2_%D0%90%D0%BD%D1%82%D0%BE%D0%BD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пакет SURPRISE:

1.  используйте данные MovieLens 1M,
2.  можно использовать любые модели из пакета,
3.  получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163752 sha256=dbe7df36bd8651e0c472b8b045b14dd4284ae7f67aec184e571832c361dab9d9
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline

#Импорт surprise
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate
%matplotlib inline

In [ ]:
!#wget  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O ml-latest.zip
##!wget https://files.grouplens.org/datasets/movielens/ml-latest.zip -O ml-latest.zip
#!unzip -o ml-latest.zip
##!rm ./ml-latest -Rf
##!mv ./ml-latest-small ./ml-latest
#prefix = 'ml-latest-small'

In [ ]:

#links = pd.read_csv(os.path.join(prefix, 'links.csv'))
#movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
#ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
#tags = pd.read_csv(os.path.join(prefix, 'tags.csv')).fillna("")
#genome_tags=pd.read_csv(os.path.join(prefix, 'genome-tags.csv'))
#genome_scores=pd.read_csv(os.path.join(prefix, 'genome-scores.csv'))

In [ ]:
#file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
#reader = Reader(line_format='user item rating timestamp', sep='\t')

#data = Dataset.load_from_file(file_path, reader=reader)


In [ ]:
data = Dataset.load_builtin('ml-100k')
#data.__dict__
data_movielens=pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])
data_movielens=data_movielens.drop(columns=['timestamp'])
data_movielens

,user_id,item_id,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
...,...,...,...
99995,880,476,3.0
99996,716,204,5.0
99997,276,1090,1.0
99998,13,225,2.0


In [ ]:
#jdata=ratings.join(movies.set_index('movieId'),on='movieId')
#jdata_scoped=jdata[['userId','title','rating']]
#jdata_scoped

In [ ]:
reader = Reader(rating_scale=(data_movielens.rating.min(), data_movielens.rating.max()))
data = Dataset.load_from_df(data_movielens, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.20)

In [ ]:


algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

RMSE: 0.9369
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9337  0.9449  0.9362  0.9364  0.9357  0.9374  0.0039  
MAE (testset)     0.7364  0.7457  0.7358  0.7388  0.7400  0.7393  0.0035  
Fit time          1.42    1.50    1.51    2.09    2.26    1.76    0.35    
Test time         0.37    0.16    0.29    0.25    0.21    0.26    0.07    


{'test_rmse': array([0.93374251, 0.94492061, 0.93621105, 0.93642015, 0.93568232]),
 'test_mae': array([0.73635854, 0.74566141, 0.73576649, 0.73877951, 0.74001613]),
 'fit_time': (1.4188835620880127,
  1.4963927268981934,
  1.5071945190429688,
  2.0934951305389404,
  2.260903835296631),
 'test_time': (0.36913609504699707,
  0.15677881240844727,
  0.29480814933776855,
  0.24789690971374512,
  0.2110276222229004)}